In [1]:
import numpy as np
import magpylib as magpy
from magpylib.magnet import Cuboid, Cylinder, CylinderSegment
from maggeometry import halbach_cylinder
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, Bounds
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid

### Nelder-Mead Adaptive Optimization of Halbach 10 main
n = 10

### Samwise

In [2]:
# def halbach_cylinder(Br, c_d, c_h, D, n, alternate=False):

In [3]:
def get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target=10, use_z=False):
    mT_to_G = 10
    B = magnets.getB(grid)
    G = B * mT_to_G
    # find magnitude of the b-field at all points over grid
    Gmag = 0
    if use_z:
        Gmag = G[:,:,2]
    else:
        Gmag = np.linalg.norm(G, axis=2)
#     Gmag = np.sqrt(G[:,:,0]**2 + G[:,:,1]**2 + G[:,:,2]**2)
#     Gmag = G[:,:,2]
    # find magnitude of b-field at center of grid
    mid_id = int(grid_res/2)
    Gcenter = Gmag[mid_id][mid_id]
    # calculate nonuniformity with respect to center b-field
    Gnon = np.abs((Gmag - Gcenter)/Gcenter)
    Gtarget_non = np.abs((Gmag - target)/target)
#     Gtarget_non = np.abs((Gmag - target)/target)
#     max_nonuniformity = np.max(Gnon)
    av_nonuniformity = np.sum(Gnon) / grid_res**2
    av_target_nonuniformity = np.sum(Gtarget_non) / grid_res**2
#     return Gmag, Gnon, Gcenter, av_nonuniformity, av_target_nonuniformity, max_nonuniformity
    return Gcenter, av_nonuniformity, av_target_nonuniformity

In [4]:
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid

In [5]:
n = 10
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
Br = 1.09e3
c_h = 640
threshold = 0.1 # in G
target = 10

          #c_d,     D
bounds = ((1, 20), (21, 300))

def obj1_nonuniformity(x):
    magnets = halbach_cylinder(Br, x[0], c_h, x[1], n)
    center_field, av_nonuniformity, av_target_nonuniformity = get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target)
    if (abs(center_field - target) > threshold):
        return av_target_nonuniformity
    return av_nonuniformity

In [6]:
options = {'disp': True, 'fatol': 1e-18, 'maxiter': 5e3, 'adaptive': True}
method = "Nelder-Mead"

shots = 25
results_x = []
results_fun = []
guesses_x0 = []
b_len = len(bounds)
for i in range(shots):
    x0 = [0, 0]
    for b in range(b_len):
        x0[b] = np.random.uniform(low=bounds[b][0], high=bounds[b][1])
    print(x0)
    res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)
    guesses_x0.append(x0)
    results_x.append(res.x)
    results_fun.append(res.fun)

print("Objective function values:")
print(results_fun)
print("Result parameters:")
print(results_x)
print("Starting guesses:")
print(guesses_x0)

[13.749130184081235, 166.74011153083745]
Optimization terminated successfully.
         Current function value: 0.000019
         Iterations: 102
         Function evaluations: 235
[14.576512684023815, 225.4106473329965]
Optimization terminated successfully.
         Current function value: 0.000027
         Iterations: 113
         Function evaluations: 261
[17.63985911764754, 158.85512370548875]


/var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_22090/1123013417.py:14: RuntimeWarning: Maximum number of iterations has been exceeded.
  res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)


[12.825099221268367, 245.19422937388998]
Optimization terminated successfully.
         Current function value: 0.000027
         Iterations: 110
         Function evaluations: 264
[1.6060780232190348, 166.49522158981864]
[15.435067067901645, 146.69628075465522]
Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 404
         Function evaluations: 857
[6.118877282432251, 227.54951169884214]
[11.864688017940983, 238.49507183755756]
Optimization terminated successfully.
         Current function value: 0.000027
         Iterations: 104
         Function evaluations: 254
[18.380728107766494, 32.87414314009353]
[15.998503939032084, 135.94768907401513]
Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 188
         Function evaluations: 428
[13.34879658826368, 28.926825974162394]
[19.308433578463923, 253.966994186805]
Optimization terminated successfully.
         Current function value: 0.000

In [7]:
print(results_x)

[array([  3.63986191, 266.22425891]), array([  4.11058208, 300.        ]), array([  3.58834144, 262.51068895]), array([  4.11046278, 300.        ]), array([ 1.28758116, 95.23744008]), array([ 1.29607843, 95.25284018]), array([ 1.28894409, 95.14789273]), array([  4.11054364, 300.        ]), array([ 1.        , 74.19345188]), array([ 1.28976262, 95.1193574 ]), array([ 1.        , 74.19342344]), array([  4.10969743, 300.        ]), array([ 1.2882745 , 95.14282064]), array([  4.10923007, 300.        ]), array([  4.10796129, 300.        ]), array([  4.11051312, 300.        ]), array([  4.10944635, 300.        ]), array([  4.11071357, 300.        ]), array([ 1.28860033, 95.17241344]), array([ 1.29082997, 95.26112591]), array([ 1.2893825 , 95.18017415]), array([  4.11053756, 300.        ]), array([ 1.29299299, 95.2249051 ]), array([  4.10942136, 300.        ]), array([  3.96183874, 289.35291477])]


In [8]:
charr = np.full((1, shots), 640).flatten()
r_n = np.full((1, shots), n).flatten()

In [9]:
from maggeometry import halbach_cylinder
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid, get_grid_mag_and_nonuniformity
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results_g_center = []
results_nonun = []
for x in results_x:
    magnets = halbach_cylinder(Br, x[0], charr[0], x[1], n)
    Gmag, Gnon, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_mag_and_nonuniformity(magnets, grid, grid_res)
    results_g_center.append(center_field)
    results_nonun.append(av_nonuniformity)

In [10]:
import pandas as pd
def make_opt_res_halbach_csv(name, fun, g_center, r_n, r_ch, x, guesses):
    r_f = np.array([fun])
    r_g = np.array(guesses)
    r_gc = np.array([g_center])
    r_n = np.array([r_n])
    r_ch = np.array([r_ch])
    r_x = np.array(x)
    a = np.concatenate((r_f.T, r_gc.T, r_n.T, r_ch.T, r_x, r_g), axis=1)
    b = a[a[:, 0].argsort()]

    columns = ['nonuniformity', 'center_field_gauss', 'n_rods', 'cyl_height']

    columns.append('cyl_diameter')
    columns.append('D')
    columns.append('g_cyl_diameter')
    columns.append('g_D')
    
    
    columns = np.array(columns)
    b_df = pd.DataFrame(b, columns = columns)
    
    c = np.concatenate((r_f.T, r_x), axis=1)
    new_array = [tuple(row) for row in c]
    u = np.unique(new_array, axis=0)
    n_unique = len(u)
    print('Number of unique results:', n_unique)
    print(b_df)
    b_df.to_csv(name + str(n_unique) + '.csv')
    return b, b_df

import datetime
today = datetime.datetime.today().strftime('%Y_%m_%d')

b, b_df = make_opt_res_halbach_csv(f'results/halbach/samwise/{n}_{today}_samwise_halbach_main_1090mT_', results_fun, results_g_center, r_n, charr, results_x, guesses_x0)

Number of unique results: 25
    nonuniformity  center_field_gauss  n_rods  cyl_height  cyl_diameter  \
0    6.274512e-07           10.046407    10.0       640.0      1.292993   
1    6.274605e-07            9.959862    10.0       640.0      1.287581   
2    6.274623e-07           10.005207    10.0       640.0      1.290830   
3    6.274724e-07            9.999760    10.0       640.0      1.288944   
4    6.274755e-07           10.088489    10.0       640.0      1.296078   
5    6.274792e-07            9.990439    10.0       640.0      1.288274   
6    6.274835e-07            9.999775    10.0       640.0      1.289382   
7    6.274882e-07            9.989276    10.0       640.0      1.288600   
8    6.275038e-07           10.018475    10.0       640.0      1.289763   
9    2.396364e-06            9.900032    10.0       640.0      1.000000   
10   2.396370e-06            9.900039    10.0       640.0      1.000000   
11   1.804764e-05           10.099997    10.0       640.0      3.588341

In [12]:
m = halbach_cylinder(Br, 1.101233, 640, 68.228286, n)
Gmag, Gnon, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_mag_and_nonuniformity(m, grid, grid_res)
print(av_nonuniformity)

1.5573750827657786e-07
